In [1]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from lagged_coherence import *
import colorednoise as cn
import scipy

In [2]:
def compute_rms(time_series):
    return np.sqrt(np.mean(np.square(time_series)))

def scale_noise(signal_without_noise, noise, desired_snr_db):
    # Compute power of the original signal
    Ps = compute_rms(signal_without_noise)**2
    
    # Convert desired SNR from dB to linear scale
    snr_linear = 10**(desired_snr_db / 10)
    
    # Calculate the desired noise power based on the desired SNR
    desired_noise_power = Ps / snr_linear
    
    # Compute scaling factor for the noise
    alpha = np.sqrt(desired_noise_power / compute_rms(noise)**2)
    
    return alpha * noise

def gen_signal_bursts(T, trials, srate, f, snr_db, num_bursts, burst_duration):
    time = np.linspace(0, T, T * srate)
    signal = np.zeros((trials, len(time)))
    total_burst_duration = num_bursts * burst_duration
    
    # Check if total burst duration is less than 90% of T
    if np.any(total_burst_duration > 0.9 * T):
        raise ValueError("Total burst duration exceeds 90% of signal duration.")
    
    # Calculate spacing between bursts
    spacing=np.zeros(num_bursts.shape)
    spacing[num_bursts>1] = (T - total_burst_duration[num_bursts>1]) / (num_bursts[num_bursts>1] - 1)    
    
    for trial in range(trials):
        # Generate burst start times without overlap
        burst_starts = np.array([i * (burst_duration[trial] + spacing[trial]) for i in range(int(num_bursts[trial]))])
        if num_bursts[trial]==1:
            burst_starts = np.array([T/2])
        
        for start in burst_starts:
            start_idx = int(start * srate)
            end_idx = start_idx + int(burst_duration[trial] * srate)
            signal_time = time[start_idx:end_idx] - start  # Resetting time for each burst
            signal[trial, start_idx:end_idx] = np.sin(2 * np.pi * f * (signal_time+np.random.randn()))
        
        # Add noise
        noise = np.random.randn(len(time))
        scaled_noise = scale_noise(signal[trial], noise, snr_db)
        signal[trial, :] += scaled_noise
        
    return signal

In [3]:
T=10
trials=100
srate=1000
snr=0

lags=np.arange(1,20.5,.5)
brst_f=np.arange(20,55,5)

f_min=10
f_max=100
n_freqs=((f_max-f_min)*2)+1

lcs_classic=np.zeros((len(brst_f), trials, n_freqs, len(lags)))*np.nan
lcs_hilbert=np.zeros((len(brst_f), trials, n_freqs, len(lags)))*np.nan
psds=np.zeros((len(brst_f), trials, n_freqs))*np.nan

brst_n=np.zeros((len(brst_f), trials))*np.nan
brst_d=np.zeros((len(brst_f), trials))*np.nan
brst_d_c=np.zeros((len(brst_f), trials))*np.nan

for f_idx, f in enumerate(brst_f):
    n=np.ones(trials)
    d_c=np.random.uniform(2, 5, size=trials)
    d=d_c/f
        
    brst_n[f_idx,:]=n
    brst_d_c[f_idx,:]=d_c
    brst_d[f_idx,:]=d

    print('{}Hz'.format(f))
    signal=gen_signal_bursts(T, trials, srate, f, snr, n, d)

    freqs, psd = scipy.signal.welch(signal, fs=srate, window='hann',
                                    nperseg=srate, noverlap=int(srate / 2), nfft=srate * 2, detrend='constant',
                                    return_onesided=True, scaling='density', axis=- 1, average='mean')
    idx=(freqs>=f_min) & (freqs<=f_max)
    freqs=freqs[idx]
    psd=psd[:,idx]
    psds[f_idx,:,:]=psd

    lcs_classic[f_idx,:,:,:]=lagged_coherence(signal, freqs, lags, srate)

    lcs_hilbert[f_idx,:,:,:]=lagged_hilbert_coherence(signal, freqs, lags, srate, n_jobs=20)
    np.savez('../output/sims/burst_dur/sim_results', 
             freqs=freqs,
             lcs_classic=lcs_classic,
             lcs_hilbert=lcs_hilbert,
             psds=psds,
             brst_n=brst_n,
             brst_d_c=brst_d_c,
             brst_d=brst_d)


20Hz
25Hz
30Hz
35Hz
40Hz
45Hz
50Hz
